Artificial Intelligence: Project 01: Taron Moe-Stull

Library Cell

In [19]:
import random, math, itertools, heapq, re, os
from collections import deque, defaultdict, Counter
from dataclasses import dataclass
from typing import Optional, List, Tuple, Dict, Set
from pathlib import Path

Parameters Cell

In [20]:
GROUP_ID = "Group30"
ALGORITHM = "ac3" # options: bt, fc, ac3, sa, ga
PUZZLE_TYPE = "evil" # options: easy, medium, hard, evil
PUZZLE_PATH = "Evil-P1.txt" # pathing

Puzzle Input and Check

In [8]:
def load_puzzle(path):
    grid = []
    with open(path, "r", encoding="utf-8-sig") as f: # was getting /ufeff error
        lines = [line.strip() for line in f if line.strip()]
    if len(lines) != 9: # invalidate puzzle if format is off
        raise ValueError(f"Expected 9 lines, got {len(lines)}")
    for i, line in enumerate(lines, start=1):
        stripped = [p.strip() for p in line.split(",")]
        if len(stripped) != 9: # invalidate puzzle if format is off
            raise ValueError(f"Line {i}: expected 9 values")
        row = []
        for p in stripped: #replacing question marks and checking
            if p == "?": 
                row.append(None)
            elif p in "123456789" and len(p) == 1:
                row.append(int(p))
            else:
                raise ValueError(f"Bad token {p!r} (use ? or 1-9)")
        grid.append(row)

    def okay(seq):
        nums = [x for x in seq if x is not None]
        return len(nums) == len(set(nums))

    for r in range(9): #row check
        if not okay(grid[r]): raise ValueError(f"Row {r} has duplicates")
    for c in range(9):#column check
        if not okay([grid[r][c] for r in range(9)]): raise ValueError(f"Col {c} has duplicates")
    for br in range(0, 9, 3): #grid check
        for bc in range(0, 9, 3):
            box = [grid[r][c] for r in range(br, br+3) for c in range(bc, bc+3)]
            if not okay(box): raise ValueError("A 3x3 grid in the puzzle has duplicates")

    return grid


Output stuff

In [9]:
def _puzzle_number_from_path(path):
    return Path(path).stem #remove .txt in file name for puzzle num in output

def save_board(board, group_id, algorithm, puzzle_type, puzzle_path): #getting output file name exact
    name = f"{group_id}_{algorithm}_{puzzle_type}_{_puzzle_number_from_path(puzzle_path)}.txt"
    with open(name, "w", encoding="utf-8") as f: #still got weird error without
        for r in range(9):
            f.write(",".join(str(board[r][c]) for c in range(9)))
            if r < 8:
                f.write("\n")
    return os.path.abspath(name)

Backtracking

In [10]:
def _row_okay(board, r, v):  return all(board[r][cc] != v for cc in range(9)) # check if nums would be duplicates in row
def _col_okay(board, c, v):  return all(board[rr][c] != v for rr in range(9)) # same but col
def _box_okay(board, r, c, v): # same but grid
    br, bc = (r//3)*3, (c//3)*3
    for rr in range(br, br+3):
        for cc in range(bc, bc+3):
            if board[rr][cc] == v:
                return False
    return True

def _is_safe_checks(board, r, c, v): # combine the three checks
    return _row_okay(board, r, v) and _col_okay(board, c, v) and _box_okay(board, r, c, v)

def _find_empty(board): # determining first empty cell
    for r in range(9):
        for c in range(9):
            if board[r][c] is None:
                return r, c
    return None

def solve_bt(board, metrics=None): 
    pos = _find_empty(board)
    if not pos:
        return True #if no empties then solved
    r, c = pos
    for v in range(1, 10): # trying nums in empties
        if _is_safe_checks(board, r, c, v): #if safe increment metric and continue checks
            if metrics is not None:
                metrics["assignments"] = metrics.get("assignments", 0) + 1 
            board[r][c] = v
            if solve_bt(board, metrics):
                return True
            board[r][c] = None
            if metrics is not None:
                metrics["backtracks"] = metrics.get("backtracks", 0) + 1
    return False

Valid Solution Check

In [11]:
def val_solution(board):
    want = set(range(1, 10)) # what every row col and grid should have
    
    for r in range(9): # row checking
        if set(board[r]) != want:
            return False
        
    for c in range(9): # col checking
        if {board[r][c] for r in range(9)} != want:
            return False
        
    for br in range(0, 9, 3): #grid checking
        for bc in range(0, 9, 3):
            box = {board[r][c] for r in range(br, br+3) for c in range(bc, bc+3)}
            if box != want:
                return False
    return True

Foreward Checking

In [12]:
def _peers(r, c): # builds sets that share rows cols or grids then removs
    ps = set()
    ps.update({(r, cc) for cc in range(9)})
    ps.update({(rr, c) for rr in range(9)})
    br, bc = (r//3)*3, (c//3)*3
    ps.update({(rr, cc) for rr in range(br, br+3) for cc in range(bc, bc+3)})
    ps.discard((r, c))
    return ps

def _init_vals(board): # changes values to given then removes given num
    domains = [[set(range(1,10)) for _ in range(9)] for __ in range(9)]
    for r in range(9):
        for c in range(9):
            if board[r][c] is not None:
                v = board[r][c]
                domains[r][c] = {v}
                for pr, pc in _peers(r,c):
                    domains[pr][pc].discard(v)
    return domains

def _fc_find_unassigned(board): # finds first empty cell
    for r in range(9):
        for c in range(9):
            if board[r][c] is None:
                return r, c
    return None

def solve_fc(board, metrics=None): #determine start vals
    domains = _init_vals(board)
    if metrics is None:
        metrics = {}

    def backtrack(): #if theres an empty cell try remaining allowed values and increment metrics
        pos = _fc_find_unassigned(board)
        if not pos:
            return True
        r, c = pos
        for v in list(domains[r][c]):
            board[r][c] = v
            old = domains[r][c]
            domains[r][c] = {v}
            if metrics is not None:
                metrics["assignments"] = metrics.get("assignments", 0) + 1
                
            pruned = [] 
            fail = False
            for rr, cc in _peers(r,c):
                if board[rr][cc] is None and v in domains[rr][cc]:
                    domains[rr][cc].remove(v)
                    pruned.append((rr, cc, v))
                    metrics["prunes"] = metrics.get("prunes", 0) + 1
                    if not domains[rr][cc]:
                        fail = True
                        break

            if not fail and backtrack():
                return True

            for rr, cc, val in pruned:
                domains[rr][cc].add(val)
            domains[r][c] = old
            board[r][c] = None
            metrics["backtracks"] = metrics.get("backtracks", 0) + 1
        return False

    return backtrack()

AC3

In [13]:
def _all_arcs(): #build list of row col grid relations
    arcs = []
    
    
    for r in range(9):
        for c in range(9):
            for rr, cc in _peers(r, c):
                arcs.append(((r, c), (rr, cc)))
                
    return arcs

def _revise(domains, Xi, Xj, metrics=None): #remove dupes and count any removal for metrics
    (ri, ci), (rj, cj) = Xi, Xj
    
    removed = False
    to_drop = set()  
    
    for x in domains[ri][ci]:
        if not any(y != x for y in domains[rj][cj]):
            to_drop.add(x)
            
    if to_drop:
        domains[ri][ci] -= to_drop 
        
        if metrics is not None:
            prunes = len(to_drop)  
            metrics["prunes"] = metrics.get("prunes", 0) + prunes 
            metrics["arc_revisions"] = metrics.get("arc_revisions", 0) + 1 
        removed = True
        
    return removed 

def _ac3(domains, metrics=None): #put arcs in queue and loop revise and return false when empty
    queue = _all_arcs()
    
    while queue:
        Xi, Xj = queue.pop(0)
        if _revise(domains, Xi, Xj, metrics):
            ri, ci = Xi
            if not domains[ri][ci]: 
                return False
            
            for rk, ck in _peers(ri, ci):
                if (rk, ck) != Xj:
                    queue.append(((rk, ck), (ri, ci)))
    return True

def _copy_domains(domains): #makes copy (can try vals easier)
    return [ [ set(domains[r][c]) for c in range(9) ] for r in range(9) ] 

def _min_rem_val(board, domains): # choose next empty cell w smallest val, reduces computation
    best = None
    
    best_size = 10 
    for r in range(9):
        for c in range(9):
            if board[r][c] is None:
                
                sz = len(domains[r][c])
                if sz < best_size:
                    best_size = sz
                    best = (r, c) 
    return best 

def solve_ac3(board, metrics=None): #finds allowed vals
    if metrics is None: 
        
        metrics = {}
    domains = _init_vals(board)
    
    if not _ac3(domains, metrics):#run once at start aand return false if broken
        return False

    def backtrack(domains): #no empties = done, find empties w mrv, try each allowed val
        pos = _min_rem_val(board, domains)
        if not pos:
            return True  
        r, c = pos
        for v in sorted(domains[r][c]): #placing val on board increments metric
            board[r][c] = v
            if metrics is not None:
                
                metrics["assignments"] = metrics.get("assignments", 0) + 1

            d2 = _copy_domains(domains)# copy allowed vals run ac3 and find constraints
            d2[r][c] = {v}
            if _ac3(d2, metrics): # if ac works and recursive solves, true
                if backtrack(d2):
                    return True

            board[r][c] = None # count backtrack try nect val
            if metrics is not None:
                metrics["backtracks"] = metrics.get("backtracks", 0) + 1
        return False

    return backtrack(domains)


SA

In [14]:
def _copy_board(b): #copies (easier to make changes) 
    
    return [row[:] for row in b]

def _givens_mask(board): #uses bools to mark where given vals are 
    
    mask = [[board[r][c] is not None for c in range(9)] for r in range(9)] 
    return mask

def _box_cells(br, bc): #returns list of coordinates
    
    return [(r, c) for r in range(br, br+3) for c in range(bc, bc+3)]

def _init_fill_by_box(board, given): #keeps givens and randomly fills blanks using remaining vals
    work = _copy_board(board)
    for br in range(0, 9, 3):
        
        for bc in range(0, 9, 3):
            cells = _box_cells(br, bc)
            
            used = {work[r][c] for (r, c) in cells if given[r][c]}
            missing = [d for d in range(1, 10) if d not in used]
            
            random.shuffle(missing)
            
            k = 0
            for (r, c) in cells:
                if not given[r][c]:
                    work[r][c] = missing[k]
                    k += 1
    return work

def _fitness_rows_cols(board): #finds duplicates using cost when 0, solved(i hope)
    
    cost = 0
    for r in range(9):
        seen = {} 
        for c in range(9):
            
            v = board[r][c]
            seen[v] = seen.get(v, 0) + 1 
        cost += sum(cnt - 1 for cnt in seen.values() if cnt > 1)
        
    for c in range(9):
        seen = {}
        for r in range(9):
            v = board[r][c]
            seen[v] = seen.get(v, 0) + 1 
        cost += sum(cnt - 1 for cnt in seen.values() if cnt > 1)
        
    return cost

def solve_sa(board, metrics=None, max_steps=20000, T0=2.0, alpha=0.999, sample_every=200):
    
    if metrics is None: 
        metrics = {}

    given = _givens_mask(board)
    work = _init_fill_by_box(board, given) #valid
    cur_fit = _fitness_rows_cols(work) 
    best = _copy_board(work)
    best_fit = cur_fit

    
    metrics["neighbor_evals"] = 0  #how many neighbors boards we checked
    metrics["moves_accepted"] = 0 #swaps that held 
    metrics["best_fitness"] = best_fit #lowest cost found
    metrics["curve"] = [(0, best_fit)] #list  of metrics for data over time

    T = T0
    
    for step in range(1, max_steps + 1): #pick random grid, find two empties, swap
        
        br, bc = (random.choice([0, 3, 6]), random.choice([0, 3, 6]))
        
        cells = [(r, c) for (r, c) in _box_cells(br, bc) if not given[r][c]]
        if len(cells) < 2: 
            continue
        (r1, c1), (r2, c2) = random.sample(cells, 2)
        
        
        work[r1][c1], work[r2][c2] = work[r2][c2], work[r1][c1] 
        new_fit = _fitness_rows_cols(work)
        metrics["neighbor_evals"] += 1  
        delta = new_fit - cur_fit

        
        accept = False # if the new board is better, pick if not calculate if worth it
        if new_fit <= cur_fit:
             
            accept = True
            
        else:
            prob = math.exp(-delta / max(T, 1e-9)) 
            accept = random.random() < prob

        if accept: # if accepted remember board and update metric
            
            metrics["moves_accepted"] += 1
            cur_fit = new_fit
            if cur_fit < best_fit:
                best_fit = cur_fit
                best = _copy_board(work) 
                metrics["best_fitness"] = best_fit
                
        else: #undo swap
            work[r1][c1], work[r2][c2] = work[r2][c2], work[r1][c1]
 
        T *= alpha

        if step % sample_every == 0:
            
            metrics["curve"].append((metrics["neighbor_evals"], best_fit))

        if best_fit == 0:
            break 

    for r in range(9): # return best
        for c in range(9): 
            
            board[r][c] = best[r][c]
    return best_fit == 0

GA

In [15]:
def _copy_board(b): return [row[:] for row in b] #make copy


def _givens_mask(board): #true where givens 
    
    return [[board[r][c] is not None for c in range(9)] for r in range(9)]

def _init_individual(board, given): #fill blanks with random
    ind = _copy_board(board)
    
    for r in range(9):
         
        used = {ind[r][c] for c in range(9) if given[r][c]} #fixed vals
        missing = [d for d in range(1,10) if d not in used] #remaining vals
        random.shuffle(missing)
        k = 0
        for c in range(9): 
            if not given[r][c]: #only fill non given
                ind[r][c] = missing[k]
                k += 1
    return ind


def _dup_count(vals): #count duplicates
    seen = {} 
    
    for v in vals:
        seen[v] = seen.get(v, 0) + 1
    return sum(cnt-1 for cnt in seen.values() if cnt > 1)

def _fitness(ind): # add dupes in cols and grids
    
    # cols
    cost = 0
    for c in range(9):
        cost += _dup_count([ind[r][c] for r in range(9)])
        
    # grids
    for br in range(0,9,3):
        for bc in range(0,9,3):
            
            box = [ind[r][c] for r in range(br, br+3) for c in range(bc, bc+3)]
            cost += _dup_count(box) 
    return cost

def _tournament(pop, fits, k=3): # pick k rand individual and return the best
    best = None
    
    best_f = 10**9
    for _ in range(k):
        
        i = random.randrange(len(pop))
        if fits[i] < best_f:
            best_f = fits[i]; best = i
    return pop[best]

def _row_swap_mutation(ind, given, rate=0.2): # swap two empties in a rand row
    if random.random() > rate:
         
        return
    r = random.randrange(9)
    free = [c for c in range(9) if not given[r][c]]
    if len(free) >= 2: 
        
        c1, c2 = random.sample(free, 2)
        ind[r][c1], ind[r][c2] = ind[r][c2], ind[r][c1]
        

def _rowwise_crossover(p1, p2, given):#in each row copy from a parent w 50/50
    child = [[0]*9 for _ in range(9)]
     
    for r in range(9):
        src = p1 if random.random() < 0.5 else p2
        child[r] = src[r][:]  #copy
    return child 

def solve_ga(board, metrics=None, pop_size=80, generations=300, mut_rate=0.2, tourn_k=3): #build population, "evolve" record metric
    if metrics is None: metrics = {}  

    given = _givens_mask(board)
    pop = [_init_individual(board, given) for _ in range(pop_size)] #starting pop
    fits = [_fitness(ind) for ind in pop]
    evals = len(fits)
    best_idx = min(range(pop_size), key=lambda i: fits[i])
    best = _copy_board(pop[best_idx]); best_f = fits[best_idx]
    
    #metrics
    metrics["fitness_evals"] = evals
    metrics["best_fitness"] = best_f
    metrics["curve"] = [(0, best_f)]
     

    for gen in range(1, generations+1): 
        
        new_pop = []
        new_pop.append(best) #keep current best
        
        while len(new_pop) < pop_size: #max out pop
             
            p1 = _tournament(pop, fits, k=tourn_k)
            p2 = _tournament(pop, fits, k=tourn_k) 
            child = _rowwise_crossover(p1, p2, given)
            _row_swap_mutation(child, given, rate=mut_rate)
            new_pop.append(child) 
            

        pop = new_pop #replace pop 
        fits = [_fitness(ind) for ind in pop] 
        evals += len(fits) 
 
        #keep track if best
        idx = min(range(pop_size), key=lambda i: fits[i])
        if fits[idx] < best_f: 
            
            best_f = fits[idx]; best = _copy_board(pop[idx])

        metrics["fitness_evals"] = evals 
        metrics["best_fitness"] = best_f 
        metrics["curve"].append((gen, best_f)) 

        if best_f == 0: #ssolved
            
            break

    for r in range(9): #return best
        for c in range(9):
            
            board[r][c] = best[r][c] 
    return best_f == 0 

Run Cell

In [21]:
board = load_puzzle(PUZZLE_PATH) 


stats = {} 

if ALGORITHM == "bt": #choose algo param
    okay = solve_bt(board, stats) 
elif ALGORITHM == "fc":
    okay = solve_fc(board, stats) 
elif ALGORITHM == "ac3":
    okay = solve_ac3(board, stats)
elif ALGORITHM == "sa":
    okay = solve_sa(board, stats)
elif ALGORITHM == "ga":
    okay = solve_ga(board, stats) 
    
else: #bad param
    
    print(f"{ALGORITHM} not found. Try bt, fc, ac3, sa, or ga (copy exactly)")
    okay = False

if okay and val_solution(board): #if success and board check passes save 
    
    out = save_board(board, GROUP_ID, ALGORITHM, PUZZLE_TYPE, PUZZLE_PATH)
    print("Solved. Saved to:", out)
    print("Metrics:", stats)
    
else: #helpful info for paper at a glance 
    if ALGORITHM in {"bt","fc","ac3","sa","ga"}:
        print(f"No valid solution with {ALGORITHM}.")
         
        if "best_fitness" in stats:
            print("Best fitness:", stats["best_fitness"])
        if "curve" in stats and stats["curve"]:
            print("Curve samples:", stats["curve"][:5], "...")

Solved. Saved to: C:\Users\Taron\Group30_ac3_evil_Evil-P1.txt
Metrics: {'assignments': 56, 'prunes': 147, 'arc_revisions': 147, 'backtracks': 1}
